<a href="https://colab.research.google.com/github/bidwatrajpokhrel/MLP-HyperparamOptimization/blob/master/7_GeneticAlgorithmCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import display

In [ ]:
#from google.colab import drive # mounts the google drive for a new notebook 
#drive.mount('/content/drive')

In [ ]:
# load the 2 npy files created by the process_yale_images.ipynb 
from numpy import load
import numpy as np
path = '/content/drive/My Drive/BidwatPhotosSample/'
# load array
y = load(path + 'yaleExtB_target.npy')
X  = load(path + 'yaleExtB_data.npy')

In [ ]:
from sklearn.model_selection import train_test_split # loads functions from the ML library sklearn 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from random import randint
from datetime import datetime

import random
from sklearn import preprocessing

In [ ]:
# split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
# PCA 
nof_prin_components = 200  # PARAMETER for optimisation in expereiments
pca = PCA(n_components=nof_prin_components, whiten=True).fit(X_train)
# applies PCA to the train and test images to calculate the principal components
X_train_pca = pca.transform(X_train) 
X_test_pca = pca.transform(X_test)
X_pca = pca.transform(X)

In [ ]:
def population_initialization(size_mlp):
    activation = ['identity','logistic', 'tanh', 'relu']
    solver = ['lbfgs','sgd', 'adam']
    learning_rate_init = [0.1,0.01,0.001]
    pop =  np.array([[random.choice(activation), random.choice(solver), random.choice(learning_rate_init), randint(1,2000)]])
    for i in range(0, size_mlp-1):
        pop = np.append(pop, [[random.choice(activation), random.choice(solver), random.choice(learning_rate_init), randint(1,2000)]], axis=0)
    return pop

In [ ]:
def crossover(parent_1, parent_2):
    child = [parent_1[0], parent_2[1], parent_1[2], parent_2[3]]    
    return child

In [ ]:
def mutation(child, prob_mut):
    lri = [0.1,0.01,0.001]
    child_ = np.copy(child)
    for c in range(0, len(child_)):
        if np.random.rand() >= prob_mut:
            child_[c,3] = int(child_[c,3]) + randint(50, 100)
            child_[c, 2] = float(random.choice(lri))
    return child_

In [ ]:
def function_fitness_mlp(pop, X,  y): 
    fitness = []
    j = 0
    for w in pop:
        clf = MLPClassifier(learning_rate_init=(float(w[2])), activation=w[0], solver = w[1], hidden_layer_sizes=(int(w[3]),),  max_iter=300, n_iter_no_change=20)

        try:
            f = cross_val_score(clf, X, y, cv=10).mean()
            fitness.append([f, clf, w])
        except:
            pass
    return fitness

In [ ]:

def genetic_mlp(X, y, num_epochs = 10, size_mlp=10, prob_mut=0.8):
    pop = population_initialization(size_mlp)
    fitness = function_fitness_mlp(pop,  X, y)
    pop_fitness_sort = np.array(list(reversed(sorted(fitness,key=lambda x: x[0]))))

    for j in range(0, num_epochs):
        length = len(pop_fitness_sort)
        #selection of parents
        parent_1 = pop_fitness_sort[:,2][:length//2]
        parent_2 = pop_fitness_sort[:,2][length//2:]

        #crossover
        child_1 = [crossover(parent_1[i], parent_2[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = [crossover(parent_2[i], parent_1[i]) for i in range(0, np.min([len(parent_2), len(parent_1)]))]
        child_2 = mutation(child_2, prob_mut)
        
        #calculates children's fitness to see if they make it to the next generation
        fitness_child_1 = function_fitness_mlp(child_1,X, y)
        fitness_child_2 = function_fitness_mlp(child_2, X, y)
        pop_fitness_sort = np.concatenate((pop_fitness_sort, fitness_child_1, fitness_child_2))
        sort = np.array(list(reversed(sorted(pop_fitness_sort,key=lambda x: x[0]))))
        
        #select individuals for next gen, drop the underpreforming ones
        pop_fitness_sort = sort[0:size_mlp, :]
        best_individual = sort[0][1]
        print(sort[0:])
        print(best_individual)

        
    return best_individual

In [ ]:
start = datetime.now()
best_result = genetic_mlp(X_pca, y, num_epochs = 20, size_mlp=10, prob_mut=0.9)
stop = datetime.now()
display.Javascript("google.colab.output.setIframeHeight('100px');")

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[0.978
  MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1781,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=20, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
  array(['tanh', 'lbfgs', '0.001', '1781'], dtype='<U5')]
 [0.9719999999999999
  MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1751,), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=20, nesterovs_momentum=True,
              power_t=0.5, r

<IPython.core.display.Javascript object>

In [ ]:
print(stop-start)

In [ ]:
print(best_result)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(1849,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=20, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


In [ ]:
cv = cross_val_score(best_result, X_pca, y, cv=10)

In [ ]:
print(cv)
print(cv.mean())

[0.95333333 1.         1.         1.         0.99333333 0.89333333
 0.97333333 1.         0.96666667 0.95333333]
0.9733333333333333
